In [21]:
import torch

In [22]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
from common.multiprocessing_env import SubprocVecEnv
from common.minipacman import MiniPacman
from common.rollout_encoder import RolloutEncoder
from common.imagination_core import ImaginationCore
from common.i2a import I2A
from common import pacman_util as putil
from common.environment_model import EnvModel

In [24]:
mode = 'regular'
env = MiniPacman(mode, 1000)
state_shape = env.observation_space.shape
num_actions = env.action_space.n
num_rewards = len(putil.task_rewards[mode])

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [25]:
full_rollout = True
env_model = EnvModel(state_shape, putil.num_pixels, num_rewards)
env_model.load_state_dict(torch.load('weights/env_model_regular'))

AssertionError: Torch not compiled with CUDA enabled